In [2]:
import pathlib

import tensorflow as tf
import tensorflow_ranking as tfr
import tensorflow_text as tf_text
import tensorflow_hub as hub
from tensorflow_serving.apis import input_pb2
from google.protobuf import text_format
from official.nlp.configs import encoders
from tensorflow_ranking.extension.premade import tfrbert_task

2023-04-18 01:33:44.957323: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-18 01:33:46.660271: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/simin/miniconda3/envs/tf-rank/lib/:/home/simin/miniconda3/envs/tf-rank/lib/python3.10/site-packages/nvidia/cudnn/lib
2023-04-18 01:33:46.660856: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home

In [25]:
# Store the paths to files containing training and test instances.
#_TRAIN_DATA_PATH = '../StackExchange/Stackoverflow/train_seq.tfrecords'
#_TEST_DATA_PATH = '../StackExchange/Stackoverflow/test_seq.tfrecords'

_TRAIN_DATA_PATH = '/tmp/train.tfrecords'
_TEST_DATA_PATH = '/tmp/test.tfrecords'

# The maximum number of documents per query in the dataset.
# Document lists are padded or truncated to this size.
_LIST_SIZE = 100

# Location of model directory and number of training steps.
_MODEL_DIR = "/tmp/tfbert_dir"
NUM_TRAIN_STEPS = 100
EVAL_STEPS = 10
batch_size = 8
SEQ_LENGTH = 128
reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]","c++","c#","f#","node.js","nodejs",".json",".js",".net","objective-c",
                                  "asp.net","ruby-on-rails","angular.js","angular-js"]

## Prepare dataset

In [3]:
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import os
import shutil
import string

In [4]:
train_path = "../StackExchange/Stackoverflow/train/"
df_train = pd.read_csv("../StackExchange/Stackoverflow/cluster_4d.csv")
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 16
seed = 42
class_list = sorted([str(folder) for folder in set(df_train["label"])])
num_class = len(class_list)

In [5]:
def remove_non_ascii(sentence):
    return ''.join(char for char in sentence if ord(char) < 128)

'''
def html_Filter(sentence):
    sentence = BeautifulSoup(sentence, "lxml").text
    #print("after html_Filter",sentence)
    
    return sentence
'''

def html_Filter(sentence):
    soup = BeautifulSoup(sentence, "html.parser")
    for data in soup(['style', 'script']):
        # Remove tags
        data.decompose()
    
    return ' '.join(soup.stripped_strings)



def keywords_transform(sentence):
    delimiter = [char for char in string.punctuation]
    punc_reserved = ['#','+','-',"'"]
    delimiter_filter = list(set(delimiter) - set(punc_reserved))
    
    sentence = sentence.lower()
    for char in delimiter_filter:
        sentence = sentence.replace(char,' '+char+' ')

    sentence = sentence.replace("node js","node.js")
    sentence = sentence.replace("node . js","node.js")
    sentence = sentence.replace(" . js",".js")
    sentence = sentence.replace(" . net",".net")
    sentence = sentence.replace("asp . net","asp.net")
    sentence = sentence.replace(" . json",".json")
    sentence = sentence.replace("objective c","objective-c")
    sentence = sentence.replace("ruby on rails","ruby-on-rails")
    sentence = sentence.replace("angular js","angular-js")
    sentence = sentence.replace("angular . js","angular.js")
    
        
    return sentence

def sent_preprocess(sentence):
    value = html_Filter(sentence)
    value = remove_non_ascii(value)
    value = keywords_transform(value)
    
    return value

In [6]:
bert_model_name = 'small_bert/bert_en_uncased_L-8_H-768_A-12' 

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


### The preprocessing model

In [37]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

In [39]:
text_test = ['this is such an amazing movie! This is not right.']
text_preprocessed = bert_preprocess_model(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

Keys       : ['input_mask', 'input_type_ids', 'input_word_ids']
Shape      : (1, 128)
Word Ids   : [ 101 2023 2003 2107 2019 6429 3185  999 2023 2003 2025 2157]
Input Mask : [1 1 1 1 1 1 1 1 1 1 1 1]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


### Create elwc dataset

In [7]:
def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

In [8]:
def create_tfexample(feature0, feature1, feature2, feature3):
    """Creates a tf.train.Example message ready to be written to a file."""
    # Create a dictionary mapping the feature name to the tf.train.Example-compatible
    # data type.
    feature = {
        'input_ids': _int64_feature(feature0),
        'input_mask': _int64_feature(feature1),
        'segment_ids': _int64_feature(feature2),
        'relevance': _int64_feature([feature3])} 
    
    # Create a Features message using tf.train.Example.

    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto

In [9]:
def create_serialized_elwc(tf_examples):
    ELWC = input_pb2.ExampleListWithContext()

    for example in tf_examples:
        example_features = ELWC.examples.add()
        example_features.CopyFrom(example)
    
    return ELWC.SerializeToString()

In [10]:
def create_dataset(dataset):
    ELWC_list = []
    bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
    
    for data in dataset:
        EXAMPLES = []
        query_preprocessed = bert_preprocess_model([data["query"]])
        query_id = query_preprocessed["input_word_ids"].numpy().flatten().tolist()
        query_mask = query_preprocessed["input_mask"].numpy().flatten().tolist()
        query_type = query_preprocessed["input_type_ids"].numpy().flatten().tolist()
        tf_query = create_tfexample(query_id, query_mask, query_type, 1)
        EXAMPLES.append(tf_query)
        
        for doc in data["documents"]:
            doc_preprocessed = bert_preprocess_model([doc["doc"]])
            input_ids = doc_preprocessed["input_word_ids"].numpy().flatten().tolist()
            input_mask = doc_preprocessed["input_mask"].numpy().flatten().tolist()
            segment_ids = doc_preprocessed["input_type_ids"].numpy().flatten().tolist()
            tf_example = create_tfexample(input_ids, input_mask, segment_ids,doc["relevance"])
            EXAMPLES.append(tf_example)
        
        ELWC_list.append(create_serialized_elwc(EXAMPLES))
    
    return ELWC_list

#### stackoverflow

In [11]:
import pandas as pd

In [12]:
df = pd.read_csv("../StackExchange/Stackoverflow/cluster_4d.csv")
df

,Id,sentence,tag,label
0,14191212,Alternative Function for EREGI in PHP <p>I was...,<php><regex><preg-match><eregi>,2867
1,14198874,JasperReports: Unsupported major.minor version...,<java><version><unsupported-class-version>,29595
2,1335851,"What does ""use strict"" do in JavaScript, and w...",<javascript><syntax><jslint><use-strict>,29590
3,1335886,mySQL Operand should contain 1 column error <p...,<mysql><sql><mysql-error-1241>,123659
4,1352885,Remove elements as you traverse a list in Pyth...,<python><list><loops><iterator><python-datamodel>,15259
...,...,...,...,...
2189,14113884,Protect C++ program against decompiling <block...,<c++><windows><obfuscation><decompiling><sourc...,29
2190,14115904,"$_POST not working. ""Notice: Undefined index: ...",<php><http-post><undefined-index>,8549
2191,14121385,Select Distinct Rows from MySQL database throu...,<php><mysql><pdo><haversine>,2802
2192,14123290,Pull data from PHP object received from twitte...,<php><twitter><twitter-streaming-api>,219989


In [13]:
label_set = set(df["label"])

In [14]:
len(label_set)

223

In [15]:
dataset = []

for label in label_set:
    df_relevant = df[df["label"] == label]    
    df_irrelevant = df[df["label"] != label]
    
    for index in list(df_relevant.index.values):
        data = {}
        data["qid"] = df_relevant.loc[index]["Id"]
        data["query"] = sent_preprocess(df_relevant.loc[index]["sentence"])
        data["label"] = df_relevant.loc[index]["label"]
        
        docs = []
        df_doc = df_relevant[df_relevant["Id"] != df_relevant.loc[index]["Id"]]
        for index_d in list(df_doc.index.values):
            doc = {}
            doc["doc_id"] = df_doc.loc[index_d]["Id"]
            doc["doc"] = sent_preprocess(df_doc.loc[index_d]["sentence"])
            doc["label"] = df_doc.loc[index_d]["label"]
            doc["relevance"] = 1
            
            docs.append(doc)
        
        
        if len(docs) < _LIST_SIZE:
            num_irrelevant = _LIST_SIZE - len(docs)
            df_irdoc = df_irrelevant.sample(num_irrelevant)
            for index_d in list(df_irdoc.index.values):
                doc = {}
                doc["doc_id"] = df_irdoc.loc[index_d]["Id"]
                doc["doc"] = sent_preprocess(df_irdoc.loc[index_d]["sentence"])
                doc["label"] = df_irdoc.loc[index_d]["label"]
                doc["relevance"] = 0
            
                docs.append(doc)
        
        data["documents"] = docs
        
        dataset.append(data)

In [17]:
#train test split
train = []
test = []
has_label = set()

for data in dataset:
    if data["label"] in has_label:
        train.append(data)
    else:
        test.append(data)
        has_label.add(data["label"])

In [18]:
print("train:",len(train))
print("test:",len(test))

train: 1971
test: 223


In [19]:
elwc_train = create_dataset(train)

2023-04-18 00:32:50.134223: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-18 00:32:50.328849: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-18 00:32:50.329455: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-18 00:32:50.330307: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, 

In [20]:
len(elwc_train)

1971

In [21]:
with tf.io.TFRecordWriter(_TRAIN_DATA_PATH) as writer:
    for objs in elwc_train:
        writer.write(objs)

In [22]:
elwc_test = create_dataset(test)

In [23]:
with tf.io.TFRecordWriter(_TEST_DATA_PATH) as writer:
    for objs in elwc_test:
        writer.write(objs)

## Create a Ranking Task for TFR-BERT

### Specify Features

In [26]:
context_feature_spec = {}
example_feature_spec = {
    'input_word_ids': tf.io.FixedLenFeature(
        shape=(SEQ_LENGTH,), dtype=tf.int64,
        default_value=[0] * SEQ_LENGTH),
    'input_mask': tf.io.FixedLenFeature(
        shape=(SEQ_LENGTH,), dtype=tf.int64,
        default_value=[0] * SEQ_LENGTH),
    'input_type_ids': tf.io.FixedLenFeature(
        shape=(SEQ_LENGTH,), dtype=tf.int64,
        default_value=[0] * SEQ_LENGTH)}
label_spec = (
    "relevance",
    tf.io.FixedLenFeature(shape=(1,), dtype=tf.int64, default_value=-1)
)

### Define Datasets

In [27]:
train_data_config = tfrbert_task.TFRBertDataConfig(
    input_path=_TRAIN_DATA_PATH,
    is_training=True,
    global_batch_size=batch_size,
    list_size=_LIST_SIZE,
    dataset_fn='tfrecord',
    seq_length=128)

validation_data_config = tfrbert_task.TFRBertDataConfig(
    input_path=_TEST_DATA_PATH,
    is_training=False,
    global_batch_size=batch_size,
    list_size=_LIST_SIZE,
    dataset_fn='tfrecord',
    seq_length=128)

### Define Task

In [32]:
# Set up task config
task_config = tfrbert_task.TFRBertConfig(
    init_checkpoint='/tmp/tfrbert/uncased_L-4_H-128_A-12/bert_model.ckpt',
    train_data=train_data_config,
    validation_data=validation_data_config,
    model=tfrbert_task.TFRBertModelConfig(
        encoder=encoders.EncoderConfig(
            bert=encoders.BertEncoderConfig(num_layers=12))))

# Set up TFRBertTask
task = tfrbert_task.TFRBertTask(
    task_config,
    label_spec=label_spec,
    dataset_fn=tf.data.TFRecordDataset,
    logging_dir=_MODEL_DIR)

## Train and evaluate

In [33]:
metrics = task.build_metrics()
model = task.build_model()
task.initialize(model)
train_dataset = task.build_inputs(task_config.train_data)
vali_dataset = task.build_inputs(task_config.validation_data)
train_iterator = iter(train_dataset)
vali_iterator = iter(vali_dataset)
optimizer = tf.keras.optimizers.Adam(lr=1e-6)

for train_step in range(NUM_TRAIN_STEPS):
    task.train_step(next(train_iterator), model, optimizer, metrics=metrics)
    train_metrics = {m.name: m.result().numpy() for m in metrics}
    print("Training metrics for epoch: " + str(train_step) + " ", train_metrics)

    if train_step % EVAL_STEPS == 0:
        task.validation_step(next(train_iterator), model, metrics=metrics)
        vali_metrics = {m.name: m.result().numpy() for m in metrics}
        print("Validation metrics for epoch: " + str(train_step) + " ",
            vali_metrics)

2023-04-18 02:03:42.458852: W tensorflow/tsl/framework/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 89.42MiB (rounded to 93763584)requested by op StatelessTruncatedNormalV2
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-04-18 02:03:42.458919: I tensorflow/tsl/framework/bfc_allocator.cc:1034] BFCAllocator dump for GPU_0_bfc
2023-04-18 02:03:42.458932: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (256): 	Total Chunks: 99, Chunks in use: 99. 24.8KiB allocated for chunks. 24.8KiB in use in bin. 657B client-requested in use in bin.
2023-04-18 02:03:42.458938: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2023-04-18 02:03:42.458943: I tensorflow/tsl/framework/

ResourceExhaustedError: {{function_node __wrapped__StatelessTruncatedNormalV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[30522,768] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:StatelessTruncatedNormalV2]
  In call to configurable 'build_encoder' (<function build_encoder at 0x7f7854427ac0>)

ow/tsl/framework/bfc_allocator.cc:1090] InUse at 7ca65d800 of size 157286400 next 479
2023-04-18 02:03:42.460506: I tensorflow/tsl/framework/bfc_allocator.cc:1090] InUse at 7d3c5d800 of size 157286400 next 481
2023-04-18 02:03:42.460509: I tensorflow/tsl/framework/bfc_allocator.cc:1090] InUse at 7dd25d800 of size 157286400 next 482
2023-04-18 02:03:42.460511: I tensorflow/tsl/framework/bfc_allocator.cc:1090] InUse at 7e685d800 of size 157286400 next 484
2023-04-18 02:03:42.460514: I tensorflow/tsl/framework/bfc_allocator.cc:1090] InUse at 7efe5d800 of size 157286400 next 485
2023-04-18 02:03:42.460516: I tensorflow/tsl/framework/bfc_allocator.cc:1090] InUse at 7f945d800 of size 157286400 next 486
2023-04-18 02:03:42.460519: I tensorflow/tsl/framework/bfc_allocator.cc:1090] InUse at 802a5d800 of size 157286400 next 487
2023-04-18 02:03:42.460521: I tensorflow/tsl/framework/bfc_allocator.cc:1090] InUse at 80c05d800 of size 157286400 next 488
2023-04-18 02:03:42.460524: I tensorflow/tsl/f

## Inference and Predictions